In [18]:
pip install minisom

In [1]:
from IPython.display import display, Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from minisom import MiniSom
import time
from hyperopt import hp,tpe,fmin,Trials, STATUS_OK

In [2]:
df=pd.read_csv('df_PCA.csv',delimiter=',')

In [3]:
df.set_index('customer_id',inplace=True)

In [4]:
x, y = max(10, int(np.ceil(np.sqrt(df.shape[0] * df.shape[1])))), max(10, int(np.ceil(np.sqrt(df.shape[0] * df.shape[1]))))
input_len=df.shape[1]

In [ ]:
space={'sig':hp.uniform('sig',0.001,x/2.01),'learning_rate':hp.uniform('learning_rate',0.001,5)}
def son_fn(space):
  sig=space['sig']
  lr=space['learning_rate']
  val=MiniSom(x=x,y=y,input_len=input_len,sigma=sig,learning_rate=lr).quantization_error(df.values)
  return {'loss': val,'status':STATUS_OK}
trials = Trials()
best=fmin(fn=son_fn,space=space,algo=tpe.suggest,max_evals=700,trials=trials)
print('best:',format(best))

  0%|          | 0/700 [00:00<?, ?trial/s, best loss=?]

In [ ]:
sigma=best['sig']
learning_rate=best['learning_rate']

In [40]:
print(sigma,learning_rate)

5.841831355518627 2.0593772582204366


In [1]:
print(x)

NameError: name 'x' is not defined

In [ ]:

sm=MiniSom(x=x,y=y,
            input_len=input_len,
            sigma=sigma,
            learning_rate=learning_rate,
            activation_distance='euclidean',
            neighborhood_function='gaussian')
sm.random_weights_init(df.values)
print(np.round(sm.quantization_error(df.values),4),"Starting QE")
sm.train_batch(df.values,10000)
print(np.round(sm.quantization_error(df.values),4),"Ending QE")